In [2]:
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/muftiwp.gov.my.jsonl

In [3]:
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
import re

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [4]:
openai.api_type = 'azure'
openai.api_base = "https://husein-ai2-aiservices.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ''

In [5]:
def predict(text):
    try:
        messages = [
            {"role": "user", "content": text},
        ]
        response = openai.ChatCompletion.create(
            engine="gpt-35-turbo-16k",
            messages=messages,
            temperature=1.0,
            max_tokens=1024,
            top_p=0.95,
        )
        return response.choices[0]['message']['content']
    except Exception as e:
        print(e)
        return None

In [7]:
data = []
with open('muftiwp.gov.my.jsonl') as fopen:
    for l in fopen:
        data.append({
            'paragraph': json.loads(l)['body']
        })
        
len(data)

3885

In [8]:
!mkdir question-muftiwp.gov.my
# !rm question-maktabahalbakri.com/*

In [9]:
def generate(filename, l):
    if os.path.exists(filename):
        return
    
    s = f"""
{l['paragraph']}

anda adalah seorang pelajar yang kurang faham tentang konteks diatas, generate one question in malay based on context above
"""
    s = s.strip()
    r = predict(s)
    l['question'] = r

    with open(filename, 'w') as fopen:
        json.dump(l, fopen)

In [11]:
generate('question-muftiwp.gov.my/0.json', data[0])

In [12]:
with open('question-muftiwp.gov.my/0.json') as fopen:
    print(json.load(fopen))

{'paragraph': 'Soalan:\nAssalamualaikum, saya ingin bertanyakan satu soalan. Apakah hukum menjual beli haiwan yang haram dimakan? Mohon penjelasan Datuk Mufti. Terima kasih.\nJawapan Ringkas\nHukum jual beli binatang yang haram dimakan adalah sah sekiranya terdapat manfaat lain seperti digunakan untuk tujuan perubatan, berburu, mengawal harta benda, dan dijadikan tunggangan, selama mana ia tidak menyalahi syarat sah jual beli yang lain. Sekiranya binatang itu tidak melepasi syarat sah jual beli iaitu tidak suci seperti babi dan anjing, tidak boleh dimanfaatkan seperti binatang buas dan binatang perosak, tidak dapat diserahkan kepada pembeli seperti burung yang terbang bebas dan tidak diketahui kondisinya, maka jual beli menjadi haram dan tidak sah.\nHuraian \nAlhamdulillah, segala puji bagi Allah SWT, selawat dan salam kepada junjungan besar Nabi Muhammad SAW, ahli keluarga Baginda SAW, sahabat Baginda SAW serta orang-orang yang mengikuti jejak langkah Baginda SAW.\nSebagaimana maklum,

In [13]:
from tqdm import tqdm

max_worker = 3
for i in tqdm(range(0, len(data), max_worker)):
    b = data[i: i + max_worker]
    filenames = [(os.path.join('question-muftiwp.gov.my', f'{i + k}.json'), l) for k, l in enumerate(b)]
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(generate, *f): f for f in filenames}

        for future in as_completed(futures):
            future.result()

  3%|█▎                                       | 40/1295 [01:46<57:05,  2.73s/it]

'content'


  7%|██▉                                      | 91/1295 [04:08<59:12,  2.95s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


  7%|██▊                                    | 95/1295 [04:23<1:08:58,  3.45s/it]

...

 98%|██████████████████████████████████████▎| 1271/1295 [59:03<01:09,  2.90s/it]

This model's maximum context length is 16384 tokens. However, your messages resulted in 17621 tokens. Please reduce the length of the messages.


100%|████████████████████████████████████▉| 1293/1295 [1:00:04<00:06,  3.01s/it]

This model's maximum context length is 16384 tokens. However, you requested 16996 tokens (15972 in the messages, 1024 in the completion). Please reduce the length of the messages or completion.


100%|█████████████████████████████████████| 1295/1295 [1:00:09<00:00,  2.79s/it]


In [15]:
from glob import glob

len(glob('question-muftiwp.gov.my/*'))

3885